In [21]:
from data_prepper.data_prepper.data_loader import DataLoader
from data_prepper.data_prepper.data_cleaner import DataCleaner
from data_prepper.data_prepper.feature_extractor import FeatureExtractor
from model_server.model_server.model import Model

In [14]:
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path='data_prepper/data_prepper/.env')

True

In [18]:
data_loader = DataLoader(entsoe_api_key=os.getenv('ENTSOE_API_KEY'))
data_loader.update_df(out_df_filepath="data/bronze/df.parquet")

In [19]:
DataCleaner.clean(
    in_df_filepath='data/bronze/df.parquet',
    out_df_filepath='data/silver/df.parquet',
)

In [20]:
FeatureExtractor.extract_features(
    in_df_filepath='data/silver/df.parquet',
    out_df_filepath='data/gold/df.parquet',
)

In [22]:
model = Model(model_filepath='data/model.joblib')
model.train(Xy_filepath='data/gold/df.parquet', n_estimators=100)

In [23]:
_, mape = model.backtest(
    Xy_filepath='data/gold/df.parquet',
    starting_ts=pd.Timestamp(datetime.now() - pd.Timedelta(30, 'd'), tz='Europe/Zurich'),
    use_every_nth_ts=100,
)
print(f'Backtested MAPE: {mape:.2f}%')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:02<00:00,  3.49it/s]

Backtested MAPE: 4.18%


In [25]:
yhat = model.predict(
    in_df_filepath='data/gold/df.parquet', 
    out_yhat_filepath='data/yhat.parquet',
)
yhat

,predicted_24h_later_load
2024-09-27 18:00:00+02:00,6672.633563
2024-09-27 19:00:00+02:00,6511.261499
2024-09-27 20:00:00+02:00,6297.196678
2024-09-27 21:00:00+02:00,6110.025362
2024-09-27 22:00:00+02:00,6218.520549
2024-09-27 23:00:00+02:00,6354.674976
2024-09-28 00:00:00+02:00,6367.178564
2024-09-28 01:00:00+02:00,6071.316424
2024-09-28 02:00:00+02:00,6324.902551
2024-09-28 03:00:00+02:00,6185.830541
